# Проект для «Викишоп»

Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

Обучите модель классифицировать комментарии на позитивные и негативные. В вашем распоряжении набор данных с разметкой о токсичности правок.

Постройте модель со значением метрики качества *F1* не меньше 0.75. 

**Инструкция по выполнению проекта**

1. Загрузите и подготовьте данные.
2. Обучите разные модели. 
3. Сделайте выводы.

Для выполнения проекта применять *BERT* необязательно, но вы можете попробовать.

**Описание данных**

Данные находятся в файле `toxic_comments.csv`. Столбец *text* в нём содержит текст комментария, а *toxic* — целевой признак.

In [1]:
!pip install spacy -q

In [2]:
import math
import re
import csv
import warnings

import pandas as pd
import numpy as np
# import spacy
from nltk import pos_tag
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
import nltk
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.pipeline import Pipeline
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score
from sklearn.naive_bayes import ComplementNB

In [3]:
pd.options.mode.chained_assignment = None
warnings.filterwarnings('ignore')

nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/jovyan/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to /home/jovyan/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [4]:
RANDOM_STATE = 1
TEST_SIZE = 0.1

## Подготовка

In [15]:
def penn2morphy(penntag):
    """ Converts Penn Treebank tags to WordNet. """
    morphy_tag = {'NN':'n', 'JJ':'a',
                  'VB':'v', 'RB':'r'}
    try:
        return morphy_tag[penntag[:2]]
    except:
        return 'n' 

In [16]:
def lemmatize_sent(text): 
    # Text input is string, returns lowercased strings.
    return [wnl.lemmatize(word.lower(), pos=penn2morphy(tag)) 
            for word, tag in pos_tag(word_tokenize(text))]

<hr>

Загрузим данные.

<hr>

In [5]:
data = pd.read_csv('/datasets/toxic_comments.csv')

In [6]:
data = data[data.columns.drop('Unnamed: 0')].reset_index(drop=True)

In [7]:
display(data.head())
data.info()

,text,toxic
0,Explanation\nWhy the edits made under my usern...,0
1,D'aww! He matches this background colour I'm s...,0
2,"Hey man, I'm really not trying to edit war. It...",0
3,"""\nMore\nI can't make any real suggestions on ...",0
4,"You, sir, are my hero. Any chance you remember...",0


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159292 entries, 0 to 159291
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   text    159292 non-null  object
 1   toxic   159292 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 2.4+ MB


<hr>

Посмотрим, есть ли дублирующиеся комментарии.

<hr>

In [8]:
data.duplicated('text').sum()

0

In [9]:
print(f'Процент токсичных комментариев: {data["toxic"].sum() / data.shape[0] * 100:.1f}%')

Процент токсичных комментариев: 10.2%


<hr>

В выборке присутствует дисбаланс классов, учтем это при разбиении на обучающую и тестовую выборки.

Лемматизируем тексты комментариев, а после разделим данные на обучающую и тестовую выборки. Далее рассчитаем TF-IDF для выборок.

<hr>

In [10]:
data['text'] = data['text'].str.lower()
data['text'] = data['text'].str.replace("[^a-zA-Z']", ' ')

In [11]:
# lemmatizer = WordNetLemmatizer()

In [12]:
# %%time

# data['lemmas'] = data['text'].apply(lambda line:
#                                     ' '.join(lemmatizer.lemmatize(word) for word in word_tokenize(line)))

In [13]:
# nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])

In [14]:
# %%time

# data['lemmas'] = data['text'].apply(lambda comment:
#                                     " ".join([token.lemma_ for token in nlp(comment)]))

In [17]:
wnl = WordNetLemmatizer()

In [18]:
%%time

data['lemmas'] = data['text'].apply(lambda comment:
                                    " ".join(lemmatize_sent(comment)))

CPU times: user 9min 11s, sys: 3.55 s, total: 9min 15s
Wall time: 9min 15s


In [19]:
data.head()

,text,toxic,lemmas
0,explanation why the edits made under my userna...,0,explanation why the edits make under my userna...
1,d'aww he matches this background colour i'm s...,0,d'aww he match this background colour i 'm see...
2,hey man i'm really not trying to edit war it...,0,hey man i 'm really not try to edit war it 's ...
3,more i can't make any real suggestions on im...,0,more i ca n't make any real suggestion on impr...
4,you sir are my hero any chance you remember...,0,you sir be my hero any chance you remember wha...


In [20]:
X_train, X_test, y_train, y_test = train_test_split(data['lemmas'], 
                                                    data['toxic'], 
                                                    random_state=RANDOM_STATE, 
                                                    test_size=TEST_SIZE, 
                                                    stratify=data['toxic'])

In [21]:
tf_idf = TfidfVectorizer(stop_words='english', lowercase=False)

In [22]:
X_train_emb = tf_idf.fit_transform(X_train)
X_test_emb = tf_idf.transform(X_test)

<hr>
<font size=4.5><b>Итоги раздела:</b></font>

<p style='text-align: justify;'>В данном разделе были загружены данные. В полученном фрейме содержится <i>159 292</i> строк и <i>2</i> столбца с комментарием и целевым признаком "токсичность", принимающим значение 1 - если комментарий токсичный, или 0 - если нет. Дублирующихся комментариев в данных нет, присутствует дисбаланс классов, соотношение токсичных комментариев к нетоксичным 1 к 9.<br><br>Данные были лемматизированы, после чего выделены и стратифицированны тренировочная и тестовая выборки и рассчитанны <i>TF-IDF</i></p>


<hr>

## Обучение

<hr>

Соберем пайплайн, а также определим наборы гиперпараметров для поиска лучших при кросс-валидации, для поиска будем использоать <i>RandomizedSearchCV</i>. Далее обучим модель, в качестве метрики будем использовать <i>f1</i>.

<hr>

In [23]:
pipe_final = Pipeline([('models', None)])

# params = [{
#              'models':[DecisionTreeClassifier(random_state=RANDOM_STATE)],
#              'models__max_depth':range(2, 50), 
#              'models__min_samples_split':range(2, 30)
#           }, 
#           {
#               'models':[KNeighborsClassifier()],
#               'models__n_neighbors':range(3, 30)
#           },
#           {
#               'models':[LogisticRegression(random_state=RANDOM_STATE, 
#                                            solver='liblinear', 
#                                            penalty='l1')],
#               'models__C':np.arange(0.5, 8, 0.5)
#           }]

params = {'models':[LogisticRegression(random_state=RANDOM_STATE, 
                                       solver='liblinear', 
                                       penalty='l1')],
          'models__C':np.arange(0.1, 10, 0.2)}

In [24]:
rsearch_cv = RandomizedSearchCV(pipe_final, 
                                params, 
                                cv=5, 
                                scoring='f1', 
                                random_state=RANDOM_STATE, 
                                n_jobs=-1, 
                                n_iter=10,
                                verbose=0)

In [25]:
%%time

_ = rsearch_cv.fit(X_train_emb, y_train);

CPU times: user 1min 58s, sys: 4.37 s, total: 2min 2s
Wall time: 2min 2s


In [26]:
print(f'Лучшая метрика f1 при валидации: {rsearch_cv.best_score_:.2f}')

Лучшая метрика f1 при валидации: 0.77


In [27]:
predictions = rsearch_cv.predict(X_test_emb)
print(f'Метрика f1 на тестовых данных: {f1_score(predictions, y_test):.2f}')

Метрика f1 на тестовых данных: 0.78


<hr>
<font size=4.5><b>Итоги раздела:</b></font>

<p style='text-align: justify;'>В данном разделе была обучена модель логистической регрессии. При обучении использовалась кросс-валидация и <i>RandomizedSearchCV</i> для поиска лучших гиперпараметров.<br>В качестве метрики использовалась <i>f1</i>. При валидации лучшая метрика получилась равной 0.77, на тестовых данных - 0.78, метрики получились близкими, значит модель не переобучилась.</p>


<hr>

## Выводы

<p style='text-align: justify;'>В данной работе была обучена модель для классификации комментариев на позитивные и негативные для интернет-магазина «Викишоп».</p>

<p style='text-align: justify;'>Сначала были загружены данные. В полученном фрейме содержится <i>159 292</i> строк и <i>2</i> столбца с комментарием и целевым признаком "токсичность", принимающим значение 1 - если комментарий токсичный, или 0 - если нет. Дублирующихся комментариев в данных нет, присутствует дисбаланс классов, соотношение токсичных комментариев к нетоксичным 1 к 9.<br><br>Данные были лемматизированы, после чего выделены и стратифицированны тренировочная и тестовая выборки и рассчитанны <i>TF-IDF</i></p>

<p style='text-align: justify;'>После чего была обучена модель логистической регрессии. При обучении использовалась кросс-валидация и <i>RandomizedSearchCV</i> для поиска лучших гиперпараметров.<br>В качестве метрики использовалась <i>f1</i>. При валидации лучшая метрика получилась равной 0.77, на тестовых данных - 0.78, метрики получились близкими, значит модель не переобучилась.</p>